In [1]:
import arcpy
import os

# Create GDB
All data is remote, but some will be processed and saved to a file GDB. Create this GDB is it doesn't exist and set this as the workspace

In [2]:
if not os.path.exists('utah.gdb'):
    arcpy.management.CreateFileGDB('.','utah.gdb')

# workspace
arcpy.env.workspace = r'./utah.gdb'
arcpy.env.overwriteOutput = True

# Define data URLs/Paths
The below cell contains all the data needed. All data is remote except **collisions_csv** and **weather_stations_csv**

In [44]:
# AADT, Intersections, Billboards
aadt_url = 'https://services1.arcgis.com/MdyCMZnX1raZ7TS3/arcgis/rest/services/Utah_AADT_2013/FeatureServer/0'
intersections_url = 'https://maps.udot.utah.gov/arcgis/rest/services/FI_Mandli2012/MapServer/5'
billboards_url = 'https://maps.udot.utah.gov/arcgis/rest/services/FI_Mandli2012/MapServer/2'

collisions_csv = r'Utah crashes_2010-2017.csv'
weather_stations_csv = 'utah_stations.csv'

# Roads data
major_roads = 'http://services1.arcgis.com/99lidPhWCzftIe9K/arcgis/rest/services/Utah_Major_Road_Centerlines_(Statewide)/FeatureServer/0'
minor_roads = 'http://services1.arcgis.com/99lidPhWCzftIe9K/arcgis/rest/services/Utah_SGID_Road_Centerlines_(Statewide)/FeatureServer/0'

# Copy the remote major roads features into local GDB

In [4]:
# Copy the major road centerlines to the gdb
_ = arcpy.management.CopyFeatures(in_features=major_roads,out_feature_class='centerlines')

In [6]:
# Join AADT
_ = arcpy.analysis.SpatialJoin('centerlines',
                           aadt_url,
                           'centerlines_aadt',
                           match_option='closest',
                           search_radius='200 feet')

                           


In [8]:
_ = arcpy.management.DeleteField('centerlines_aadt',['route_id','Shape_Leng','OBJECTID_12'])
_ = arcpy.management.AlterField('centerlines_aadt','aadt_vn','aadt',clear_field_alias=True)

# Merge major/minor roads

In [9]:
# Merge minor roads.
_ = arcpy.management.Merge(['centerlines_aadt',minor_roads],'centerlines_merged')

# Rename/Delete Fields
For consistancy, readabiliy of the Python code, let's rename some of the fields and delete ones we don't care about

In [10]:
print([f.name for f in arcpy.ListFields('centerlines_merged')])

['OBJECTID_1', 'Shape', 'Join_Count', 'TARGET_FID', 'OBJECTID', 'ADDR_SYS', 'CARTOCODE', 'FULLNAME', 'L_F_ADD', 'L_T_ADD', 'R_F_ADD', 'R_T_ADD', 'PREDIR', 'STREETNAME', 'STREETTYPE', 'SUFDIR', 'ALIAS1', 'ALIAS1TYPE', 'ALIAS2', 'ALIAS2TYPE', 'ACSALIAS', 'ACSNAME', 'ACSSUF', 'ADDR_QUAD', 'USPS_PLACE', 'ZIPLEFT', 'ZIPRIGHT', 'COFIPS', 'ONEWAY', 'SPEED', 'VERTLEVEL', 'CLASS', 'HWYNAME', 'DOT_RTNAME', 'DOT_RTPART', 'DOT_F_MILE', 'DOT_T_MILE', 'MODIFYDATE', 'GLOBALID', 'COLLDATE', 'ACCURACY', 'SOURCE', 'NOTES', 'COUNIQUE', 'SURFTYPE', 'SURFWIDTH', 'DSTRBWIDTH', 'LOCALFUNC', 'MAINTJURIS', 'FED_RDID', 'STATUS', 'ACCESS', 'USAGENOTES', 'DOT_RTID', 'DOT_FUNC', 'DOT_COFUND', 'LOCALID', 'aadt', 'Shape_Length']


In [11]:
# Fields we don't need in the centerlines dataset for this
delete_fields = [
    'ADDR_SYS',
    'CARTOCODE',
    'FULLNAME',
    'L_F_ADD',
    'L_T_ADD',
    'R_F_ADD',
    'R_T_ADD',
    'STREETNAME',
    'ALIAS1',
    'ALIAS1TYPE',
    'ALIAS2',
    'ALIAS2TYPE',
    'ACSALIAS',
    'ACSNAME',
    'ACSSUF',
    'ADDR_QUAD',
    'USPS_PLACE', 
    'ZIPLEFT', 
    'ZIPRIGHT', 
    'COFIPS',
    'VERTLEVEL', 
    'CLASS', 
    'HWYNAME', 
    'DOT_RTNAME',
    'DOT_RTPART',
    'DOT_F_MILE', 
    'DOT_T_MILE', 
    'MODIFYDATE',
    'GLOBALID', 
    'COLLDATE', 
    'ACCURACY', 
    'SOURCE', 
    'NOTES', 
    'COUNIQUE', 
    'DSTRBWIDTH', 
    'LOCALFUNC',
    'MAINTJURIS', 
    'FED_RDID', 
    'STATUS', 
    'ACCESS', 
    'USAGENOTES',
    'DOT_RTID',
    'DOT_FUNC', 
    'DOT_COFUND',
    'LOCALID'
]
field_remap = {
    'PREDIR':'pre_dir',
    'STREETTYPE':'street_type',
    'SUFDIR':'suf_dir',
    'ONEWAY':'one_way',
    'SPEED':'speed_limit',
    'SURFTYPE':'surface_type',
    'SURFWIDTH':'surface_width'
}
_ = arcpy.management.DeleteField('centerlines_merged',delete_fields)
for field_name,remap_name in field_remap.items():
    arcpy.management.AlterField('centerlines_merged',field_name,remap_name,clear_field_alias=True)

In [14]:
print([f.name for f in arcpy.ListFields('centerlines_merged')])

['OBJECTID_1', 'Shape', 'Join_Count', 'TARGET_FID', 'OBJECTID', 'pre_dir', 'street_type', 'suf_dir', 'one_way', 'speed_limit', 'surface_type', 'surface_width', 'aadt', 'Shape_Length']


We will reference *segment_id* as the unique identifier for a road segment

In [15]:
# Add a segment_id field to use instead of OBJECTID_1
_ = arcpy.management.AddField('centerlines_merged','segment_id','Long')
_ = arcpy.management.CalculateField('centerlines_merged','segment_id','!OBJECTID_1!')

# Calculate Fields
There are several fields to add to the data to enrich. Some will be calculated off of the geometries, some off of proximity to features in other datasets

In [17]:
# Now we add some calculated fields:

fields = [
    ['sinuosity','Double'],
    ['euclidean_length','Double'],
    ['segment_length','Double'],
    ['at_intersection','Short'],
    ['near_billboard','Short'],
    ['road_orient_approx','Double'],
    ['near_major_road','Short']
]

_ = arcpy.management.AddFields('centerlines_merged',fields)

<Result './utah.gdb\\centerlines_merged'>

In [19]:
# Calc Sinuosity
code_block = \
'''
import math
def getSinuosity(shp):
    x0 = shp.firstPoint.x
    y0 = shp.firstPoint.y
    x1 = shp.lastPoint.x
    y1 = shp.lastPoint.y

    euclid = math.sqrt((x0-x1)**2 + (y0-y1)**2)
    length = shp.length
    if euclid > 0:
        return length/euclid
    return 1.0
'''
_ = arcpy.management.CalculateField('centerlines_merged','sinuosity','getSinuosity(!Shape!)',code_block=code_block)

In [20]:
# Calc Euclidean Length
code_block = \
'''
import math
def getEuclideanLength(shp):
    x0 = shp.firstPoint.x
    y0 = shp.firstPoint.y
    x1 = shp.lastPoint.x
    y1 = shp.lastPoint.y

    euclid = math.sqrt((x0-x1)**2 + (y0-y1)**2)
    return euclid
'''
_ = arcpy.management.CalculateField('centerlines_merged','euclidean_length','getEuclideanLength(!Shape!)',code_block=code_block)

In [21]:
# Calc segment_length
_ = arcpy.management.CalculateField('centerlines_merged','segment_length','!Shape_Length!')

In [22]:
# Calc Road Orientation
code_block = \
'''
import math
def getOrientation(shp):
    x0 = shp.firstPoint.x
    y0 = shp.firstPoint.y
    x1 = shp.lastPoint.x
    y1 = shp.lastPoint.y

    angle = math.atan2(y1-y0,x1-x0)
    if angle < 0:
        angle += math.pi
    return angle
'''
_ = arcpy.management.CalculateField('centerlines_merged','road_orient_approx','getOrientation(!Shape!)',code_block=code_block)

In [24]:
# Calc at_intersection
_ = arcpy.analysis.Near('centerlines_merged',
                        intersections_url,
                        search_radius='150 feet')
_ = arcpy.management.CalculateField('centerlines_merged','at_intersection','!NEAR_DIST! > 0')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [25]:
# Calc near_billboard
_ = arcpy.analysis.Near('centerlines_merged',
                        billboards_url,
                        search_radius='300 feet')
_ = arcpy.management.CalculateField('centerlines_merged','near_billboard','!NEAR_DIST! > 0')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [27]:
# Calc near_major_road
_ = arcpy.analysis.Near('centerlines_merged',
                        'centerlines',
                        search_radius='50 feet')
_ = arcpy.management.CalculateField('centerlines_merged','near_major_road','!NEAR_DIST! > 0')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [66]:
# Load collisions

# Projected Coords UTM NAD1983 Zone 12N
in_sr = arcpy.SpatialReference(26912)

_ = arcpy.management.MakeXYEventLayer(collisions_csv,
        out_layer='collisions',
        in_x_field='Longitude',
        in_y_field='Latitude',
        spatial_reference=in_sr)

# Projected Coords Web Mercator
out_sr = arcpy.Describe(static_features).spatialReference

_ = arcpy.management.Project('collisions','collisions_projected',out_sr)

# Weather Station Polygons
We need to create the Thiessen polygons for the weather station

In [57]:
in_sr = arcpy.SpatialReference(4326)
_ = arcpy.management.MakeXYEventLayer(weather_stations_csv,
        out_layer='weather_station_eventlayer',
        in_x_field='LON',
        in_y_field='LAT',
        spatial_reference=in_sr)
res = arcpy.management.CopyFeatures('weather_station_eventlayer','weather_station_locations',)
_ = arcpy.analysis.CreateThiessenPolygons(res,'weather_station_polygons',fields_to_copy='ALL')

Merge weather station polygons with the roads

In [77]:
field_mappings = arcpy.FieldMappings()
field_mappings.addTable('centerlines_merged')
features_mapping = arcpy.FieldMap()
features_mapping.addInputField('weather_station_polygons','station_id')
field_mappings.addFieldMap(features_mapping)

_ = arcpy.analysis.SpatialJoin('centerlines_merged',
                               'weather_station_polygons',
                               'centerlines_merged_with_weather',
                               join_type='KEEP_ALL',
                               match_option='closest',
                               search_radius='100 miles',
                               field_mapping=field_mappings)

In [78]:
_ = arcpy.management.CopyFeatures('centerlines_merged_with_weather','static_features')

# Join Collisions to Road Features

In [76]:
field_mappings = arcpy.FieldMappings()
field_mappings.addTable('collisions_projected')

features_mapping = arcpy.FieldMap()
features_mapping.addInputField('static_features','segment_id')
field_mappings.addFieldMap(features_mapping)

_ = arcpy.analysis.SpatialJoin('collisions_projected',
                               'static_features',
                               'collisions_joined',
                               join_type='KEEP_COMMON', # Right join
                               match_option='closest',
                               search_radius='50 feet',
                               field_mapping=field_mappings)